In [ ]:
# pip installs

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.4 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 peft==0.14.0 trl==0.14.0 fsspec==2025.3.0 matplotlib wandb datasets

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
HF_USER = "Vishy08"
PROJECT_NAME = "product-pricer"

In [ ]:
LOG_TO_WANDB = True
RUN_NAME = f"{datetime.now() :%d-%m-%Y_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

In [ ]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token , add_to_git_credential = True)

wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ['WANDB_API_KEY'] = wandb_api_key
wandb.login()

In [ ]:
os.environ["WANDB_PROJECT"] = 'product-pricer'
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

In [ ]:

QUANT_4_BIT = True

if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit = True ,
        bnb_4bit_use_double_quant = True ,
        bnb_4bit_compute_dtype = torch.bfloat16 ,
        bnb_4bit_quant_type = 'nf4' ,
    )
else :
    quant_config = BitsAndBytesConfig(
        load_in_8bit = True ,
        bnb_8bit_compute_dtype = torch.bfloat16,
    )

In [ ]:

#tokenizer
BASE_MODEL = 'meta-llama/Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL , trust_remote_code = True )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL  ,
    quantization_config = quant_config ,
    device_map = 'auto'
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(base_model)

In [ ]:
dataset = load_dataset("Vishy08/pricer-data")

In [ ]:
from transformers import TrainerCallback
import torch
import random

class TestPromptCallback(TrainerCallback):
    def __init__(self, tokenizer, val_dataset, num_samples=2):
        self.tokenizer = tokenizer
        self.val_dataset = val_dataset
        self.num_samples = num_samples

    def on_evaluate(self, args, state, control, **kwargs):
        # This triggers whenever evaluation runs (e.g. every 50 or 100 steps)
        print(f"\n--- VISUAL CHECK at Step {state.global_step} ---")

        # Select random samples from the validation set
        indices = random.sample(range(len(self.val_dataset)), self.num_samples)
        model = kwargs['model']
        model.eval() # Switch to evaluation mode

        for idx in indices:
            sample = self.val_dataset[idx]
            # Use 'test_prompt' which has NO answer (e.g. "... Price is $")
            input_text = sample['test_prompt']

            inputs = self.tokenizer(input_text, return_tensors="pt").to(model.device)

            with torch.no_grad():
                # Generate up to 10 tokens (enough for a price)
                outputs = model.generate(**inputs, max_new_tokens=10, pad_token_id=self.tokenizer.pad_token_id)

            # Decode the result
            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            prediction = generated_text.replace(input_text, "").strip()

            print(f"Sample {idx}:")
            print(f"Input:     {input_text.strip()} [ends here]")
            print(f"Predicted: {prediction}")
            print(f"Actual:    {sample['price']}") # Compare with hidden truth
            print("-" * 30)

        model.train() # Switch back to training mode

# Initialize the callback with your validation set
# We will pass this to the trainer in the next block
printer_callback = TestPromptCallback(tokenizer, dataset['val'])

In [ ]:

train_subset = dataset['train'].shuffle(seed=42).select(range(100000))
val_dataset = dataset['val']

print(f"Train on: {len(train_subset)}")
print(f"Val on: {len(val_dataset)}")

In [ ]:
#QLORA Parameters
LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

In [ ]:
#hyperparameter
EPOCHS = 1
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

In [ ]:
STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
run = wandb.init(project = "product-pricer" , resume = "allow" , id = "nxo7668d")


In [ ]:
artifact = run.use_artifact("llm_engineering/product-pricer/model-nxo7668d:v2",type ="model")
artifact_dir = artifact.download()

In [ ]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# LoRA Config
lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# SFT Config
train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=1,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=LEARNING_RATE,
    logging_steps=50,


    report_to = 'wandb',

    # Validation Settings
    eval_strategy="steps",
    eval_steps=200,
    do_eval=True,

    # Dataset Settings
    max_seq_length= 1024,
    dataset_text_field="prompt",
    packing=False,

    # Hub / Saving
    save_steps=500,
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)

In [ ]:
#INITIALIZATION
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train_subset,
    eval_dataset=val_dataset,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator,
    callbacks=[printer_callback]
)

In [ ]:
# Point to the folder shown in your file explorer screenshot
checkpoint_path = "/content/artifacts/model-nxo7668d:v2"

In [ ]:
fine_tuning.train(resume_from_checkpoint=checkpoint_path)